In [1]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 6.0 MB/s eta 0:00:00a 0:00:01


In [2]:
import torch
from torch.utils.data import DataLoader
import math
import pandas as pd
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-07 09:21:07.744572: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-07 09:21:07.744669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-07 09:21:07.920594: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv('/kaggle/input/sentence-similarity-nepali-dataset/stsb_multi_mt_nepali_cleaned.csv')
df.head()

,sentence1,sentence2,score
0,एउटा विमान उडिरहेको छ।,हवाई जहाज उडिरहेको छ।,5.00
1,एउटा मान्छे ठूलो बाँसुरी बजाइरहेको छ।,एउटा मान्छे बाँसुरी बजाउँदै छ।,3.80
2,एक व्यक्ति पिज्जामा टुक्रा चिज फैलाउँदै छ।,एक जना मानिसले न पकाएको पिज्जामा टुक्रा पारेको...,3.80
3,तीन जना चेस खेल्दै छन्।,दुई जना पुरुष चेस खेलिरहेका छन्।,2.60
4,एउटा मान्छे सेलो बजाउँदै छ।,बसेको मान्छे सेलो खेलिरहेको छ।,4.25


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
model_name = 'Rajan/NepaliBERT'

train_batch_size = 16
num_epochs = 100
model_save_path = '/kaggle/working/sentence_transformer_nepali_retrained'

In [6]:
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False,
                               )

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/328M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertModel were not initialized from the model checkpoint at Rajan/NepaliBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/987k [00:00<?, ?B/s]

In [7]:
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [8]:
input_example_samples = []

for index, row in df.iterrows():
    score = float(row['score']) / 5.0 # Normalize score between 0 to 1
    inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=score)

    input_example_samples.append(inp_example)

In [9]:
len(input_example_samples)

5749

In [10]:
import random

random.shuffle(input_example_samples)

train_ratio = 0.8
test_ratio = 0.1
dev_ratio = 0.1

num_examples = len(input_example_samples)
num_train = int(num_examples * train_ratio)
num_dev = int(num_examples * dev_ratio)
num_test = int(num_examples * test_ratio)


train_samples = input_example_samples[:num_train]
dev_samples = input_example_samples[num_train:num_train + num_dev]
test_samples = input_example_samples[num_train + num_dev:]

In [11]:
print("Train samples:", len(train_samples))
print("Dev samples:", len(dev_samples))
print("Test samples:", len(test_samples))

Train samples: 4599
Dev samples: 574
Test samples: 576


In [12]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size = train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

In [13]:
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='stsb-dev-nepali')

In [14]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) # 10% of train data for warm-up

In [15]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator = evaluator,
          epochs = num_epochs,
          evaluation_steps = 1000,
          warmup_steps = warmup_steps,
          output_path = model_save_path
)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Stsb-dev-nepali Pearson Cosine,Stsb-dev-nepali Spearman Cosine,Stsb-dev-nepali Pearson Manhattan,Stsb-dev-nepali Spearman Manhattan,Stsb-dev-nepali Pearson Euclidean,Stsb-dev-nepali Spearman Euclidean,Stsb-dev-nepali Pearson Dot,Stsb-dev-nepali Spearman Dot,Stsb-dev-nepali Pearson Max,Stsb-dev-nepali Spearman Max
288,No log,No log,0.584433,0.529995,0.570037,0.535467,0.570009,0.534883,0.404444,0.411658,0.584433,0.535467
576,0.072300,No log,0.630269,0.579386,0.590948,0.558463,0.590836,0.557443,0.509211,0.491773,0.630269,0.579386
864,0.072300,No log,0.658447,0.610811,0.612098,0.578173,0.612644,0.578573,0.547548,0.528451,0.658447,0.610811
1000,0.047000,No log,0.666375,0.614651,0.626983,0.590599,0.627172,0.590727,0.562621,0.535285,0.666375,0.614651
1152,0.047000,No log,0.672237,0.625900,0.627099,0.590611,0.628047,0.590636,0.560009,0.540695,0.672237,0.625900
1440,0.047000,No log,0.680891,0.635564,0.637430,0.598613,0.637586,0.598777,0.557042,0.541733,0.680891,0.635564
1728,0.034000,No log,0.672592,0.632945,0.637961,0.598947,0.637936,0.598995,0.550762,0.539140,0.672592,0.632945
2000,0.021700,No log,0.672088,0.637508,0.638572,0.600721,0.638595,0.600175,0.562049,0.553229,0.672088,0.637508
2016,0.021700,No log,0.675216,0.638240,0.637745,0.599568,0.637940,0.599418,0.567959,0.554272,0.675216,0.638240
2304,0.021700,No log,0.687561,0.646783,0.645779,0.608069,0.645841,0.607783,0.583634,0.568408,0.687561,0.646783


Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

...

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

In [16]:
sentences_2 = [
    "सरकारले कपास विकास समिति खारेज गर्ने निर्णय गरेको सुनेर मलाई दुःख लाग्यो। नेपालमा कपास खेतीको राम्रो सम्भावना र बजार दुवै छ। यसको उत्पादन त हामीले बाउ-बाजेका पालादेखि नै गर्दै आएका हौं। र अहिले पनि धेरै किसानले आफ्नो प्रयोगका लागि पनि कपास खेती गर्दै आएका छन्।व्यावसायिक रूपमा सुरु गरिएको कपास खेती सरकारको गलत नीतिका कारण आज बन्द हुने अवस्थामा पुगेको हो। समिति खारेज भएपछि कपास उत्पादनका लागि २०३३ सालदेखि गरिएका सबै प्रयास खेर गए। मैले कृषि प्राविधिकको रूपमा आफ्नो जागिरे जीवन सुरु गरेर १५-१६ वर्ष कपास खेतीकै क्षेत्रमा बिताए। पछि कृषि सचिव भएर पनि एक वर्षभन्दा बढी काम गरें।",
    "पार्टीको जिल्ला नेतृत्वले पार्टी सुधारको मागलाई बेवास्ता गरेको भन्दै नेकपा (एमाले) सिद्धार्थनगर नगर कमिटीका सचिवसहित ७४ जनाले सामूहिक राजीनामा दिएका छन् । सोमबार भैरहवामा पत्रकार सम्मेलन गरी नगर सचिव नारायणप्रसाद भण्डारीसहित नगर कमिटी र विभिन्न जनवर्गीय संगठनका पदाधिकारीले राजीनामा दिएको घोषणा गरेका हुन् । पत्रकार सम्मेलनमा बोल्दै भण्डारीले एक महिनाअघि पार्टीमा गर्नुपर्ने सुधारको माग राख्दै नेतृत्वलाई १० बुँदे मागसहित सुझाव पत्र पेस गरिएको तर जिल्ला नेतृत्वले त्यसलाई बेवास्ता गरी उल्टै व्यक्तिगत लाञ्छना र कारबाहीको धम्की दिंदै गुटगत सोचले अघि बढेपछि राजीनामा दिनुपरेको बताए ।"
]

embeddings_2 =model.encode(sentences_2)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim_2 = cosine_similarity(
    [embeddings_2[1]],
    [embeddings_2[1]]
)

cos_sim_2

array([[1.]], dtype=float32)

In [18]:
!pip install huggingface_hub

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
from huggingface_hub import login
access_token_write = "your_access_token"
login(token = access_token_write)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [25]:
model.push_to_hub('syubraj/sentenceTransformer_nepali_new')

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

'https://huggingface.co/syubraj/sentenceTransformer_nepali_new/commit/70099c0437a80b82a5644295e3a327e1558fbeca'